In [66]:
#!pip install yfinance pymoo

In [67]:
import yfinance as yf
import numpy as np
import pandas as pd
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.decomposition.asf import ASF
from pymoo.optimize import minimize
import matplotlib
matplotlib.use('TkAgg')
from datetime import datetime
import pandas as pd
import numpy as np

In [68]:
def save_results_to_csv(res, weight_return, weight_risk, weight_diversification, file_name_prefix="portfolio_solutions"):
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"output/{file_name_prefix}_{current_time}.csv"

    solutions = []
    for i, (f, x) in enumerate(zip(res.F, res.X)):
        # Normaliza as alocações para que somem exatamente 1
        x_normalized = x / np.sum(x)  # Normaliza para que a soma das alocações seja 1
        x_normalized = np.round(x_normalized, 4)
        x_normalized = x_normalized / np.sum(x_normalized)


        score = (weight_return * f[0]) - (weight_risk * f[1]) + (weight_diversification * f[2])

        allocations_str = ', '.join([f"{alloc:.4f}" for alloc in x_normalized]).strip()

        solutions.append({
            "ID": i + 1,
            "Allocations": allocations_str, 
            "Return": f[0],
            "Risk": f[1],
            "Diversification": f[2],
            "Score": score
        })
    

    df = pd.DataFrame(solutions)

    df.to_csv(file_name, index=False)
    print(f"Soluções salvas em {file_name} com sucesso!")

In [69]:
def optimize_portfolio(tickers=["AAPL", "MSFT", "GOOGL", "AMZN"],
                       start_date="2015-01-01",
                       end_date="2024-01-01",
                       pop_size=100,
                       n_gen=1000,
                       total_invested=1,
                       max_risk=0.1,
                       weight_return=1,
                       weight_risk=1,
                       weight_diversification=1):
    
    data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    returns = data.pct_change().dropna()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()

    class PortfolioOptimizationProblem(ElementwiseProblem):
        def __init__(self):
            super().__init__(n_var=len(mean_returns), n_obj=3, n_constr=2, xl=0, xu=1)

        def _evaluate(self, x, out, *args, **kwargs):
            x = x / np.sum(x)

            # Restrições
            g1 = np.sum(x) - total_invested  # A soma das alocações deve ser igual ao total investido
            g2 = np.dot(x.T, np.dot(cov_matrix, x)) - max_risk  # O risco (variância) do portfólio deve ser menor ou igual ao risco máximo permitido

            # Funções objetivo
            f1 = np.dot(x, mean_returns)  # Maximizar o retorno esperado
            f2 = np.dot(x.T, np.dot(cov_matrix, x))  # Minimizar o risco (variância)
            f3 = -np.sum(x * np.log(x))  # Maximizar a diversificação (entropia)

            # Resultado das funções objetivo normalizadas
            out["F"] = [f1, f2, f3]
            out["G"] = [g1, g2]

    algorithm = NSGA2(pop_size=pop_size)
    problem = PortfolioOptimizationProblem()
    res = minimize(problem, algorithm, termination=('n_gen', n_gen), save_history=True)

    best_solution = None
    best_score = -np.inf

    print("Melhores soluções encontradas:")
    for i, (f, x) in enumerate(zip(res.F, res.X)):
        x_normalized = x / np.sum(x)
        
        x_normalized = np.round(x_normalized, 4)
        x_normalized = x_normalized / np.sum(x_normalized)

        # Calcular o score composto baseado nos 3 objetivos
        score = (weight_return * f[0]) - (weight_risk * f[1]) + (weight_diversification * f[2])

        if score > best_score:
            best_score = score
            best_solution = x_normalized
            best_return = f[0]
            best_risk = f[1]
            best_diversification = f[2]

        print(f"Solução {i+1}: Índice: {i+1}, Alocações: {x_normalized * 100}, "
              f"Retorno: {f[0]}, Risco: {f[1]}, Diversificação: {f[2]}, Score: {score}")

    print("\nMelhor solução baseada nos três objetivos:")
    print(f"Alocações: {best_solution * 100}")
    print(f"Retorno: {best_return}, Risco: {best_risk}, Diversificação: {best_diversification}")
    print(f"Score composto: {best_score}")

    save_results_to_csv(res, weight_return, weight_risk, weight_diversification)
    print("Matriz de covariância:\n", cov_matrix)
    print("Valores diagonais (variâncias):", np.diag(cov_matrix))
    return res


In [70]:
res1 = optimize_portfolio(
    tickers=["AAPL", "MSFT", "GOOGL", "AMZN"],
    start_date="2015-01-01",
    end_date="2024-01-01",
    pop_size=50,
    n_gen=100,
    total_invested=1,
    max_risk=1,
    weight_return=1,
    weight_risk=1,
    weight_diversification=1
)


[*********************100%***********************]  4 of 4 completed


Melhores soluções encontradas:
Solução 1: Índice: 1, Alocações: [  0.   0. 100.   0.], Retorno: 0.000896154843524795, Risco: 0.0003228115541466216, Diversificação: 0.0001218822005892249, Score: 0.0006952254899673983
Solução 2: Índice: 2, Alocações: [30.86  8.27 30.58 30.29], Retorno: 0.0010538556042476049, Risco: 0.00025170812630770806, Diversificação: 1.293037336514302, Score: 1.2938394839922418
Solução 3: Índice: 3, Alocações: [30.63306331  8.89088909 31.84318432 28.63286329], Retorno: 0.001051494701300451, Risco: 0.000251753662310072, Diversificação: 1.3001130225446902, Score: 1.3009127635836806
Solução 4: Índice: 4, Alocações: [ 0.          4.75047505  0.         95.24952495], Retorno: 0.0011434403652499177, Risco: 0.0003015746812936754, Diversificação: 0.19133866132797733, Score: 0.19218052701193358
Solução 5: Índice: 5, Alocações: [12.63  0.   87.24  0.13], Retorno: 0.0009196072671382797, Risco: 0.0002967872234014933, Diversificação: 0.38906769158661, Score: 0.38969051163034674
S

In [71]:
res2 = optimize_portfolio(
    tickers=["AAPL", "MSFT", "GOOGL", "AMZN"],
    start_date="2015-01-01",
    end_date="2024-01-01",
    pop_size=50,
    n_gen=100,
    total_invested=1,
    max_risk=1,
    weight_return=1,
    weight_risk=1,
    weight_diversification=0
)


[*********************100%***********************]  4 of 4 completed


Melhores soluções encontradas:
Solução 1: Índice: 1, Alocações: [  0.   0. 100.   0.], Retorno: 0.0008961540976019088, Risco: 0.00032281168696898316, Diversificação: 0.00010388644502322273, Score: 0.0005733424106329256
Solução 2: Índice: 2, Alocações: [30.97  8.13 30.86 30.04], Retorno: 0.0010529809613801123, Risco: 0.0002517074711258133, Diversificação: 1.2911209458485424, Score: 0.0008012734902542989
Solução 3: Índice: 3, Alocações: [7.200e-01 2.000e-02 1.100e-01 9.915e+01], Retorno: 0.0011384634683082484, Risco: 0.0003058501026656711, Diversificação: 0.053583572168868075, Score: 0.0008326133656425774
Solução 4: Índice: 4, Alocações: [4.500e-01 2.000e-02 1.728e+01 8.225e+01], Retorno: 0.001096910385007994, Risco: 0.00028458169319651587, Diversificação: 0.4903101654989549, Score: 0.0008123286918114781
Solução 5: Índice: 5, Alocações: [2.000e-02 0.000e+00 9.870e+00 9.011e+01], Retorno: 0.001115153292424764, Risco: 0.0002934063828577248, Diversificação: 0.32388118649345166, Score: 0.000

In [72]:
res3 = optimize_portfolio(
    tickers=["AAPL", "MSFT", "GOOGL", "AMZN"],
    start_date="2015-01-01",
    end_date="2024-01-01",
    pop_size=50,
    n_gen=100,
    total_invested=1,
    max_risk=1,
    weight_return=1,
    weight_risk=0,
    weight_diversification=0
)


[*********************100%***********************]  4 of 4 completed


Melhores soluções encontradas:
Solução 1: Índice: 1, Alocações: [31.06310631  8.24082408 30.57305731 30.1230123 ], Retorno: 0.0010537270009028075, Risco: 0.000251707793765871, Diversificação: 1.2926097823342506, Score: 0.0010537270009028075
Solução 2: Índice: 2, Alocações: [28.34283428  9.6109611  34.32343234 27.72277228], Retorno: 0.0010474868636998085, Risco: 0.00025205381134060063, Diversificação: 1.3051846033107548, Score: 0.0010474868636998085
Solução 3: Índice: 3, Alocações: [1.150e+00 5.290e+00 8.000e-02 9.348e+01], Retorno: 0.0011430493220477918, Risco: 0.0002990656627614381, Diversificação: 0.27539814824840647, Score: 0.0011430493220477918
Solução 4: Índice: 4, Alocações: [0.000e+00 0.000e+00 9.999e+01 1.000e-02], Retorno: 0.0008961697581047322, Risco: 0.0003228005239589352, Diversificação: 0.0007623398133589127, Score: 0.0008961697581047322
Solução 5: Índice: 5, Alocações: [32.04  0.   67.57  0.39], Retorno: 0.0009557909072861503, Risco: 0.0002722635450803785, Diversificação: